$$\textbf{PLN. Práctica 3}$$
$$\textit{Y. Sarahi García Gozález}$$

In [1]:
import re
import os

In [ ]:
def get_text_from_file(path_corpus,path_truth):

    tr_txt=[]
    tr_y=[]

    with open(path_corpus, "r") as f_corpus,open(path_truth, "r") as f_truth:
        for url in urls_conferencias:
            f_salida.write(url + "\n")
